In [1]:
from IPython.display import Image
from IPython.core.display import HTML

In [2]:
import matplotlib
matplotlib.use("Agg")

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

Using TensorFlow backend.


In [4]:
dataset = "images"
modelname = "santa_pepe_binary_crossentropy_dropout"
plot = "plot_binary_crossentropy_dropout.png"

In [5]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 32

In [6]:
data = []
labels = []
labels_name = ["not_santa","pepe","santa"]
print(labels_name[0])

not_santa


In [7]:
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)
print(imagePaths)

['images\\not_santa\\00000028.jpg', 'images\\not_santa\\00000456.jpg', 'images\\santa\\00000065.jpg', 'images\\santa\\00000237.jpg', 'images\\santa\\00000400.jpg', 'images\\santa\\00000389.jpg', 'images\\not_santa\\00000345.jpg', 'images\\not_santa\\00000241.jpg', 'images\\not_santa\\00000221.jpg', 'images\\santa\\00000130.jpg', 'images\\santa\\00000116.jpg', 'images\\santa\\00000307.jpg', 'images\\not_santa\\00000354.jpg', 'images\\not_santa\\00000287.jpg', 'images\\santa\\00000162.jpg', 'images\\santa\\00000267.jpg', 'images\\not_santa\\00000350.jpg', 'images\\not_santa\\00000049.jpg', 'images\\santa\\00000407.jpg', 'images\\santa\\00000120.jpg', 'images\\not_santa\\00000036.jpg', 'images\\santa\\00000054.jpg', 'images\\santa\\00000072.jpg', 'images\\santa\\00000397.jpg', 'images\\santa\\00000051.jpg', 'images\\not_santa\\00000110.jpg', 'images\\santa\\00000258.jpg', 'images\\santa\\00000100.jpg', 'images\\santa\\00000041.jpg', 'images\\not_santa\\00000105.jpg', 'images\\santa\\00000

In [8]:
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]

    for i in range(len(labels_name)):
        if label == labels_name[i] :
            label = i
            
    labels.append(label)

In [9]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [10]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

In [11]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=3)
testY = to_categorical(testY, num_classes=3)

In [12]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

Start Building the Model Right Now:

In [13]:
from keras import models
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout

In [14]:
model = models.Sequential()

In [15]:
height = 28
width = 28
depth = 3
classes = len(labels_name)
inputShape = (height, width, depth)

First set of CONV => RELU => POOL layers

In [16]:
model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

Second set of CONV => RELU => POOL layers

In [17]:
model.add(Conv2D(50, (5, 5), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

First (and only) set of FC => RELU layers

In [18]:
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(500))
model.add(Activation("relu"))

Softmax classifier

In [19]:
model.add(Dense(classes))
model.add(Activation("softmax"))

In [20]:
Image(url="https://live.staticflickr.com/65535/48238891246_092637c136_z.jpg")

In [21]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

In [22]:
#model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [23]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

Epoch 1/50
21/21 [==============================] - 2s 79ms/step - loss: 0.4675 - acc: 0.7188 - val_loss: 0.3565 - val_acc: 0.8290
Epoch 2/50
21/21 [==============================] - 1s 53ms/step - loss: 0.3509 - acc: 0.8423 - val_loss: 0.3427 - val_acc: 0.8578
Epoch 3/50
21/21 [==============================] - 1s 58ms/step - loss: 0.2444 - acc: 0.9090 - val_loss: 0.1953 - val_acc: 0.9282
Epoch 4/50
21/21 [==============================] - 1s 52ms/step - loss: 0.2055 - acc: 0.9278 - val_loss: 0.2647 - val_acc: 0.9167
Epoch 5/50
21/21 [==============================] - 1s 53ms/step - loss: 0.1886 - acc: 0.9288 - val_loss: 0.2457 - val_acc: 0.9195
Epoch 6/50
21/21 [==============================] - 1s 51ms/step - loss: 0.1547 - acc: 0.9425 - val_loss: 0.2303 - val_acc: 0.9152
Epoch 7/50
21/21 [==============================] - 1s 54ms/step - loss: 0.1450 - acc: 0.9459 - val_loss: 0.2135 - val_acc: 0.9339
Epoch 8/50
21/21 [==============================] - 1s 54ms/step - loss: 0.1231 - a

In [24]:
model.save(str(modelname) + ".h5")

In [25]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Handsomeness")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)

Result of the accuracy using categorical_crossentropy (loss)

In [26]:
Image(url="https://live.staticflickr.com/65535/48239450162_104d4895e7_z.jpg")

Result of the accuracy using binary_crossentropy (loss)

In [27]:
Image(url="https://live.staticflickr.com/65535/48239450192_4d10ef4f12_z.jpg")